In [1]:
from tda.client import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime

import seaborn as sns

import login.config as config
from login.client import TDSession

In [29]:
def wol( trig_time, profit_ratio, loss_ratio, df, i):  # win or loss?  Ps the trig_price is the open price of the next minute
    # find time to start searching original df
    i = i+1
    j = i+1
    trig_price = df.iloc[i, 0]
    df = df.iloc[j:,:] 

    # compare prices
    for i,row in df.iterrows():
        # row[2] corresponds to low min price
        if row[2] >= trig_price * profit_ratio:
            return 'profit'     # indicates a sell/profit
        # row[1] corresponds to high min price
        elif row[1] <= trig_price * loss_ratio:
            return 'loss'     # indicates a sell/loss
        else:
            continue

#With the WOL function - create a comprehension that will then be used as a column to indicate win or loss

def wol_time( trig_time, profit_ratio, loss_ratio, df, i):  # win or loss?  Ps the trig_price is the open price of the next minute
    # find time to start searching original df
    i = i+1
    j = i+1
    trig_price = df.iloc[i, 0]
    df = df.iloc[j:,:] 

    # compare prices
    for i,row in df.iterrows():
        # compare to low of min row[2]
        if row[2] >= trig_price * profit_ratio:
            return (row[5] - trig_time)/60000     # indicates a sell/profit
        # compare to high of min row[1]
        elif row[1] <= trig_price * loss_ratio:
            return (row[5] - trig_time)/60000      # indicates a sell/loss
        else:
            continue

In [63]:
symbols = ["LXRX", "FCEL", "NNDM", "DPW", "MARA", "AMD", "CCL", "RIOT", "DDD", "MRO", "FUBO", "TSLA", "OXY", "BBBY", "GEVO", "PBR", "M", "MVIS", "NCLH", "UAVS", "ET", "SRNE", "FCX", "X", "SOLO", "MAC", "SUNW", "FTFT", "DVN", "FTI", "JWN", "CLF", "APA", "BLNK", "BLDP", "SPWR", "SKT", "SLB", "CVE", "PBF", "HAL", "MIK", "RRC", "W", "WKHS", "AR", "MPC", "KODK", "CNK", "HL", "AMRN", "SABR", "SQ", "COTY", "MGM", "MGNI", "OSTK", "NK", "SM", "CBAT", "ENG", "RUN", "RCL", "BCRX", "KNDI", "PRTY", "PBI", "VLO", "NOV", "PENN", "CLSK", "CEMI", "CNQ", "PBR.A", "VUZI", "AA", "NVTA", "CXO", "OVV", "MGY", "EOG", "KSS", "PAA", "PEIX", "CYH", "HOME", "OKE", "WPRT", "MUR", "MT", "RESN", "CPE", "SOL", "GRWG", "IGT", "BLDR", "PK", "FANG", "EVRI", "AYRO"]
# symbols = ['PBR', 'CBAT', 'KODK', 'RRC', 'CCL', 'BCRX', 'RUN', 'SRNE', 'COTY',
#        'ET', 'RCL', 'CEMI', 'CYH', 'CXO', 'CVE', 'NK', 'SQ', 'EOG',
#        'PBR.A', 'FANG', 'ENG', 'SPWR', 'UAVS', 'AYRO','INN', 'CELH', 'KIRK', 'DAN', 'VVPR', 'ELYS', 'FTEK', 'YRCW',
#        'CPRI', 'CUK', 'FUV', 'BGFV', 'HES', 'STKL', 'TTM', 'APTV', 'WYNN',
#        'PGEN', 'WNDW', 'TGTX', 'TRGP', 'GT', 'BLMN', 'APPS', 'CRSP', 'DBI', 'LNC', 'PLYA', 'CZR', 'PEB']
symbols = symbols[:4]

In [64]:
days = 10
sym_dict = []
real_roi = list()

# loop through chosen symbols
# symbols chosen by high volume and high beta
for i,symbol in enumerate(symbols):
    
    # Get the price history iteratively
    data = TDSession.get_price_history(
        symbol, 
        period_type=Client.PriceHistory.PeriodType.DAY,
        period=Client.PriceHistory.Period.TEN_DAYS,
        frequency_type=Client.PriceHistory.FrequencyType.MINUTE,
        frequency=Client.PriceHistory.Frequency.EVERY_MINUTE)
    data = data.json()

    
    # Create dataframe with new features
    test = pd.DataFrame(data['candles'])
    test['time'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%I:%M:%S"),axis=1 )
    test['day'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%d"),axis=1 )
    test['month'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%B"),axis=1 )
    #     test['year'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%Y"),axis=1 )
    #create open to high column
    test['open_high'] = test['high']-test['open']
    #create open to low column
    test['open_low'] = test['open']-test['low']
    
    
    # Define the conditions for the trigger
    #define variables
    candle_diff = 1.2
    morn15 = ['08:{}:00'.format(30+x) for x in range(0,15)]

    # --------------------------------------------------condition for trigger-----------------------------------
    #define open higher than open low
    high_not_zero = test['open_high'] != 0 
    low_not_zero = test['open_low'] != 0 
    tall_candle = test['open_high'] > candle_diff*test['open_low']
    volume75 = test['volume'] >= np.percentile(test['volume'],[30])[0] #90
    #     volume75 = test['volume'] <= np.percentile(test['volume'],[10])[0] #90
    not_first15 = ~test['time'].isin(morn15)
    price65th = test['open']<np.percentile(test['high'],[15])[0] #65
    #     down_trend = past_high_mean < recent_low_mean * 1.02
    #     price35th = test['open']>np.percentile(test['high'],[45])[0]

    condition = high_not_zero & low_not_zero & tall_candle & volume75 & not_first15 & price65th#  & price35th & down_trend

    
    
    
    # Checking for trigger, store in trigger_df
    trigger_df = test.loc[condition, :].reset_index()

    
    
    
    #add column of desired sell prices and loss prices
    #desired roi is 0.2%
    
    
    #    _____________________________________________-    ROI    -________________________________________________
    
    roi = 0.01
    
    
    my_profit = 1+roi
    trigger_df['to_sell_price'] = trigger_df.open * my_profit
    
    my_loss = 1-(roi/2)
    trigger_df['to_loss_price'] = trigger_df.open * my_loss
    
    trigger_df['wol'] = [ wol( row[6], my_profit, my_loss, test, row[0]) for i,row in trigger_df.iterrows() ]
    
    trigger_df['wol_time'] = [ wol_time( row[6], my_profit, my_loss, test, row[0]) for i,row in trigger_df.iterrows() ]
    
    
    
    print(symbol,i)
    print(trigger_df.wol.value_counts())
    
    
    ind_roi = round((profit*(my_profit-1))-(loss*(1-my_loss)),5)

    ind_time = trigger_df.wol_time.mean()
    #     print('Average trade time in minutes: ',ind_time)

    profit = trigger_df.loc[trigger_df['wol']=='profit','wol'].count()
    loss = trigger_df.loc[trigger_df['wol']=='loss','wol'].count()

    sym_data = {
        'symbol':symbol,
        'total_sells':profit,
        'total_loss':loss,
        'min_to_sell':round(ind_time,4)
    }

    # append data to appropriate lists
    sym_dict.append(sym_data)
    real_roi.append((symbol, ind_roi, ind_time, trigger_df.iloc[0,4]))

    
    
    
trades = pd.DataFrame.from_dict(sym_dict, orient='columns')
tradables = list()
tickers = list()
price = list()

# print(real_roi)
for x in real_roi:
    if x[2] < 120:
        tradables.append(x[1])
        tickers.append(x[0])
        price.append(x[3])

print( sum(tradables) )
print(tradables)
print(tickers)

LXRX 0
profit    19
loss      19
Name: wol, dtype: int64
FCEL 1
loss      149
profit    126
Name: wol, dtype: int64
NNDM 2
loss      115
profit     66
Name: wol, dtype: int64
DPW 3
loss      83
profit    45
Name: wol, dtype: int64
0.79
[0.095, 0.095, 0.515, 0.085]
['LXRX', 'FCEL', 'NNDM', 'DPW']


In [65]:
trades

,symbol,total_sells,total_loss,min_to_sell,profit
0,LXRX,19,19,27.3421,1.01
1,FCEL,126,149,29.4364,1.01
2,NNDM,66,115,30.6575,1.01
3,DPW,45,83,18.8047,1.01


In [48]:
tradables = list()
tickers = list()
price = list()
for x in real_roi:
    if x[2] < 120 and x[3] > 2:
        tradables.append(x[1])
        tickers.append(x[0])
        price.append(x[3])

print( sum(tradables) )
print(tickers)


1.5950000000000002
['LXRX', 'FCEL', 'NNDM', 'DPW', 'MARA', 'RIOT']


In [66]:
perc_prof = trades.loc[trades['min_to_sell']<60, :].sort_values(by='min_to_sell').total_sells.sum() * (my_profit-1)
perc_los = trades.loc[trades['min_to_sell']<60, :].sort_values(by='min_to_sell').total_loss.sum() * (1-my_loss)
perc_prof - perc_los
trades.loc[trades['min_to_sell']<60, :].sort_values(by='min_to_sell').symbol.values

array(['DPW', 'LXRX', 'FCEL', 'NNDM'], dtype=object)